In [10]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [11]:
# device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
# hyper parameters
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epoches = 2
learning_rate = 0.01

In [13]:
# mnist dataset
train_dataset = torchvision.datasets.MNIST(root='./mnist', 
                                           train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist', 
                                           train=False, transform=transforms.ToTensor())
# data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [14]:
# rnn (many to one)
class RNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        # set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # forward propagate lstm
        out, _ = self.lstm(x, (h0, c0)) # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        # decode the hidden state of the last time step
        out = self.fc(out[:,  -1,  :])
        
        return out

In [15]:
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

In [16]:
# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [17]:
# training
total_step = len(train_loader)
for epoch in range(num_epoches):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i + 1) % 100 == 0:
            print('epoch {} / {}, step {} / {}, loss {:.4f}'.format(epoch + 1, num_epoches, i + 1, total_step, loss.item()))

epoch 1 / 2, step 100 / 600, loss 0.7106
epoch 1 / 2, step 200 / 600, loss 0.4668
epoch 1 / 2, step 300 / 600, loss 0.2414
epoch 1 / 2, step 400 / 600, loss 0.0818
epoch 1 / 2, step 500 / 600, loss 0.0818
epoch 1 / 2, step 600 / 600, loss 0.1211
epoch 2 / 2, step 100 / 600, loss 0.0831
epoch 2 / 2, step 200 / 600, loss 0.0718
epoch 2 / 2, step 300 / 600, loss 0.0561
epoch 2 / 2, step 400 / 600, loss 0.2113
epoch 2 / 2, step 500 / 600, loss 0.0810
epoch 2 / 2, step 600 / 600, loss 0.0953


In [18]:
# testing
with torch.no_grad():
    correct = 0
    total = 0
    
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('test accuracy of the model on the 10000 test images: {} %'.format(correct / total * 100))

test accuracy of the model on the 10000 test images: 97.52 %


In [19]:
# save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')